In [27]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from repo_funciones import *


In [28]:
censo=pd.read_csv('../DATA/censo_clean.csv', encoding='latin-1')

In [29]:
type(censo)

pandas.core.frame.DataFrame

In [30]:
col_unique_counts(censo, 'fuente')

,fuente,Count
0,No especificado,32
1,Del servicio público de agua,16
2,De un pozo comunitario,16
3,De un pozo particular,16
4,De una pipa,16
5,De otra vivienda,16
6,De la lluvia,16
7,De otro lugar,16
8,Acarreada de un pozo,16
9,Acarreada de llave comunitaria,16


In [40]:
censo.loc[censo['fuente']=='De una pipa' , 'fuente']='Camion cisterna'
censo.loc[censo['fuente']=='La trae una pipa' , 'fuente']='Camion cisterna'
censo.loc[censo['fuente']=='Acarreada de otra vivienda' , 'fuente']='De otra vivienda'
censo.loc[censo['fuente']=='La recolectan de la lluvia' , 'fuente']='De la lluvia'
censo.loc[censo['fuente']=='De otro lugar' , 'fuente']='No especificado'
censo.loc[censo['fuente']=='Acarreada de un pozo' , 'fuente']='De un pozo comunitario'
censo.loc[censo['fuente']=='Acarreada de llave comunitaria' , 'fuente']='Llave o pozo comunitario'
censo.loc[censo['fuente']=='De un pozo comunitario' , 'fuente']='Llave o pozo comunitario'

In [47]:
censo.loc[censo['fuente']=='De una pipa' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='Camion cisterna' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De otra vivienda' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De la lluvia' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='No especificado' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='Llave o pozo comunitario' , 'disponibilidad']='no'
censo.loc[censo['fuente']=='De un pozo particular' , 'disponibilidad']='no'

In [49]:
censo_group=censo.groupby(['alcaldia','disponibilidad', 'fuente'])['poblacion'].sum()


In [50]:
censo_grouped=pd.DataFrame(censo_group)

In [53]:
censo_grouped.head(10)

poblacion
alcaldia      disponibilidad fuente                                 
Azcapotzalco  no             Camion cisterna                    2345
                             De la lluvia                         15
                             De otra vivienda                    293
                             De un pozo particular                65
                             Llave o pozo comunitario            860
                             No especificado                     525
              si             Del servicio público de agua     426697
Benito Juárez no             Camion cisterna                     360
                             De la lluvia                          8
                             De otra vivienda                     67

In [54]:
censo_grouped.to_csv('../DATA/censo_grouped.csv')